In [49]:
from music21 import *
#configure.run()

import os

#testing directory access
cwd = os.getcwd() #getting current directory
dataset_dir = os.path.join(cwd,'Classical-Piano-Composer-master/Classical-Piano-Composer-master/midi_songs/bla')
print('dataset_dir: '+dataset_dir)
print("cwd:" +cwd)
print(os.listdir(dataset_dir))

notes = []

#funziona, ora devo capire perchè sotto esplode
s = converter.parse(r'C:\Users\Mattia\Desktop\Polimi\Magistrale\Creative Computing\TrapGenerator\Classical-Piano-Composer-master\Classical-Piano-Composer-master\midi_songs\bla\test.mid')
print(s)
s.show('text')
#s.show()
s_part=instrument.partitionByInstrument(s)
#print(len(s_part))
#notes=s_part.parts[0].recurse()
print(notes)
notes = s.flat.notes

for element in notes:
    if isinstance(element,chord.Chord):
        #print(str(element.pitch))
        print("chord duration: "+element.duration.type)
    if isinstance(element,note.Note):
        print("note duration: "+element.duration.type)

        

print(s_part)

#print(len(s_part))

#s_part.parts[2] #prendo il mio music object, ne calcolo le parti (strumenti presenti) e lo salvo nella variabile s_part.
#dopo con .parts[n] seleziono la parte n-esima. Quindi ad esempio con 0 seleziono il pianoforte, con 1 il basso, 2 la voce, ecc....
#Nel codice seguente uso sempre .parts[0] perchè così seleziono il pianoforte in ogni caso 

dataset_dir: C:\Users\Mattia\Desktop\Polimi\Magistrale\Creative Computing\TrapGenerator\Classical-Piano-Composer-master/Classical-Piano-Composer-master/midi_songs/bla
cwd:C:\Users\Mattia\Desktop\Polimi\Magistrale\Creative Computing\TrapGenerator
['test.mid']
<music21.stream.Score 0x1f9fa06e888>
{0.0} <music21.stream.Part 0x1f9fc8da7c8>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.chord.Chord C3 E3 G3>
    {1.0} <music21.note.Note C>
    {3.0} <music21.note.Note E>
    {4.0} <music21.chord.Chord C3 E3 G3>
    {5.0} <music21.note.Note C>
    {7.0} <music21.note.Rest rest>
    {8.0} <music21.chord.Chord D3 F#3 A3>
    {16.0} <music21.chord.Chord F3 A3 C4>
    {24.0} <music21.chord.Chord F3 A3 C4>
[]
chord duration: half
note duration: half
note duration: half
chord duration: quarter
note duration: half
chord duration: breve
chord duration: breve
chord duration: breve
None


In [34]:
#load full data
notes = []

for file in os.listdir(dataset_dir):
    print('sto printando file: '+ file)
    midi = converter.parse(os.path.join(dataset_dir,file)) #metodo per convertire il midi in formato music21
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi) #prende il file midi e vede se è composto da più tracce, cioè strumenti
    #print(len(parts.parts))
    if parts: # file has instrument parts. Condizione imposta sul "truthy" cioè basta che abbia un valore diverso da 0 o null
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

sto printando file: test.mid


In [35]:
print(notes) #nel printing i valori numerici puntati (ad esempio 4.9) indicano un accordo. I singoli numeri indicano la nota
#ad esempio lo 0 è un DO, il 2 è un RE e così via, fino a 12.


['0.4.7', '2.6.9', '5.9.0', '5.9.0']


In [ ]:
sequence_length = 100
# get all pitch names
pitchnames = sorted(set(item for item in notes))

# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)